In [69]:
import os
import sys
home_dir = "../../"
module_path = os.path.abspath(os.path.join(home_dir))
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd

In [77]:
dms_filename = "TPMT_HUMAN_Matreyek_2018" # PTEN_HUMAN_Matreyek_2021, TPMT_HUMAN_Matreyek_2018
dms_variants_df = pd.read_csv(home_dir+f"data/dms/ProteinGym_substitutions/{dms_filename}.csv", sep=",")
dms_variants_df

,mutant,mutated_sequence,DMS_score,DMS_score_bin
0,M1G,GDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.027709,1
1,M1T,TDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.006009,1
2,M1S,SDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.913619,1
3,M1R,RDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.017972,1
4,M1Q,QDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.223651,1
...,...,...,...,...
3643,K245G,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.883663,1
3644,K245F,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.147061,1
3645,K245E,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.970427,1
3646,K245A,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.942823,1


In [78]:
# extracting dbnsfp scores
dbnsfp_scores_df = pd.read_csv(home_dir+"models/dbnsfp/outputs_dms/dbNSFP44a_TMPT.tsv", sep="\t") # dbNSFP44a_TMPT, dbNSFP44a_PTEN

cols = ['aaref', 'aaalt', 'aapos', 'SIFT_score', 'Polyphen2_HVAR_score', 'MetaRNN_score', 'REVEL_score', 'MVP_score',
       'CADD_raw', 'integrated_fitCons_score', 'phyloP17way_primate', 'phastCons17way_primate', 'bStatistic']
dbnsfp_scores_df = dbnsfp_scores_df[cols]
dbnsfp_scores_df = dbnsfp_scores_df[(dbnsfp_scores_df['aaref'] != ".") & (dbnsfp_scores_df['aaalt'] != ".")]
dbnsfp_scores_df.drop_duplicates(keep="first", inplace=True, ignore_index=True)

x = []
for i in range(dbnsfp_scores_df.shape[0]):
    row_dict = dict(dbnsfp_scores_df.loc[i])
    # print(row_dict)
    mut_positions = str(row_dict["aapos"]).split(";")
    # if len(mut_positions) > 1:
    for mut_pos in mut_positions:
        row_dict["aapos"] = str(mut_pos).strip()
        x.append(row_dict)
    # if i==5: break

dbnsfp_scores_df = pd.DataFrame.from_dict(x)
dbnsfp_scores_df["mutant"] = dbnsfp_scores_df["aaref"] + dbnsfp_scores_df["aapos"].apply(lambda x: str(x)) + dbnsfp_scores_df["aaalt"]
dbnsfp_scores_df.drop(columns=['aaref', 'aaalt', 'aapos'], inplace=True)
dbnsfp_scores_df.drop_duplicates(keep="first", inplace=True, ignore_index=True)
dbnsfp_scores_df["mutant"].value_counts()

# dbnsfp_scores_df[dbnsfp_scores_df["mutant"] == "D77E"]
# dbnsfp_scores_df

merged_df = dms_variants_df.merge(dbnsfp_scores_df, on=["mutant"], how="left")
merged_df


,mutant,mutated_sequence,DMS_score,DMS_score_bin,SIFT_score,Polyphen2_HVAR_score,MetaRNN_score,REVEL_score,MVP_score,CADD_raw,integrated_fitCons_score,phyloP17way_primate,phastCons17way_primate,bStatistic
0,M1G,GDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.027709,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M1T,TDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.006009,1,0.0,0.838,0.9830154,0.663,0.88945481203,2.922758,0.706298,0.756,0.494,751.0
2,M1S,SDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.913619,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M1R,RDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.017972,1,0.0,0.928,0.98916423,0.674,0.905578272566,3.356390,0.706298,0.756,0.494,751.0
4,M1Q,QDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.223651,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3790,K245G,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.883663,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3791,K245F,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.147061,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3792,K245E,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.970427,1,0.003,0.838,0.5206004,0.498,0.762845975942,3.787910,0.732398,0.609,0.953,753.0
3793,K245A,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.942823,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
all_methods = [("esm_rives","esm1b_t33_650M_UR50S"), ("esm_rives", "esm1v_t33_650M_UR90S"), ("esm_rives", "esm2_t33_650M_UR50D"), 
                   ("bioembeddings_dallago", "prottrans_bert_bfd"), ("bioembeddings_dallago", "prottrans_albert_bfd"), ("bioembeddings_dallago", "plus_rnn"),
                   ("rostlab_huggingface", "prottrans_t5_xl_u50"), 
                   ("vespa_marquet", "vespa"), 
                   ("proteinbert_brandes", "proteinbert"), ("sequnet_dunham", "sequnet"), 
                   ("tape_rao", "protbert"),
                   ("jax_unirep", "unirep")]
task = "dms"                   

In [80]:
# merged_df = dms_variants_df.copy()
for method in all_methods:
    model_root, model_name = method

    models_pred_df = pd.read_csv(home_dir+f"models/{model_root}/outputs/{model_name}/{task}/preds_{dms_filename}.tsv", sep="\t")
    models_pred_df.drop(
        columns=[
            "mutated_sequence",
            "DMS_score",
            "DMS_score_bin"
        ],
        inplace=True,
    )
    
    merged_df = pd.merge(left=merged_df, right=models_pred_df, how="left", on=["mutant"])
    merged_df.rename(columns={"pred": f"{model_name}_score"}, inplace=True)
    print(model_root, model_name, merged_df.shape)
    # break

esm_rives esm1b_t33_650M_UR50S (3795, 15)
esm_rives esm1v_t33_650M_UR90S (3795, 16)
esm_rives esm2_t33_650M_UR50D (3795, 17)
bioembeddings_dallago prottrans_bert_bfd (3795, 18)
bioembeddings_dallago prottrans_albert_bfd (3795, 19)
bioembeddings_dallago plus_rnn (3795, 20)
rostlab_huggingface prottrans_t5_xl_u50 (3795, 21)
vespa_marquet vespa (3795, 23)
proteinbert_brandes proteinbert (3795, 24)
sequnet_dunham sequnet (3795, 25)
tape_rao protbert (3795, 26)
jax_unirep unirep (3795, 27)


In [81]:
merged_df

,mutant,mutated_sequence,DMS_score,DMS_score_bin,SIFT_score,Polyphen2_HVAR_score,MetaRNN_score,REVEL_score,MVP_score,CADD_raw,...,prottrans_bert_bfd_score,prottrans_albert_bfd_score,plus_rnn_score,prottrans_t5_xl_u50_score,vespa_pred,vespal_pred,proteinbert_score,sequnet_score,protbert_score,unirep_score
0,M1G,GDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.027709,1,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.138005,-7.081413,-3.294519,-6.320984,0.533993,0.562116,-10.896611,-0.998386,-0.612508,0.610085
1,M1T,TDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.006009,1,0.0,0.838,0.9830154,0.663,0.88945481203,2.922758,...,-5.462499,-7.175518,-2.966695,-6.047627,0.466776,0.472452,-11.829899,-0.998387,-0.122962,0.649223
2,M1S,SDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.913619,1,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.813991,-6.989775,-2.365203,-5.529820,0.457978,0.472452,-11.620841,-0.998295,0.271600,0.655836
3,M1R,RDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.017972,1,0.0,0.928,0.98916423,0.674,0.905578272566,3.356390,...,-5.240901,-7.102805,-2.965577,-6.129631,0.468171,0.472452,-12.520124,-0.998339,-0.182586,0.559217
4,M1Q,QDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.223651,1,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.004263,-7.498725,-3.577618,-6.558220,0.444359,0.427919,-12.883346,-0.998383,-0.312214,0.553580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3790,K245G,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.883663,1,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.618822,-1.636718,-4.816307,-5.529600,0.456200,0.478569,-7.360335,-0.971417,-1.567277,0.018396
3791,K245F,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,1.147061,1,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.823155,-2.978440,-5.580591,-6.063119,0.496530,0.523547,-7.964895,-0.971635,-2.417926,0.021755
3792,K245E,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.970427,1,0.003,0.838,0.5206004,0.498,0.762845975942,3.787910,...,-0.663941,-0.753745,-4.594479,-4.398368,0.347777,0.348442,-6.976328,-0.969022,-1.251635,0.013143
3793,K245A,MDGTRTSLDIEEYSDTEVQKNQVLTLEEWQDKWVNGKTAFHQEQGH...,0.942823,1,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.289828,-1.720250,-5.184930,-4.079212,0.401293,0.433933,-7.625202,-0.970485,-1.700961,0.015386


In [82]:
merged_df.to_csv(home_dir+f"data/merged_dms_predictions/{dms_filename}.tsv", sep="\t", header=True, index=False)